In [ ]:
import numpy as np
import scipy.io
import mne
import ipywidgets
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse
import matplotlib.pyplot as plt


# 1. load MATLAB data
mat_data = scipy.io.loadmat('name.mat')
eeg_data = mat_data['data']eeg_data = np.transpose(eeg_data, (2, 0, 1)) 
print(f"data shape: {eeg_data.shape} (channel×timepoints×trails)")

# 2. Create the MNE data structure
sfreq = 500  tmin = 0  
ch_names = [
    'Fp1', 'Fp2', 
    'AF3', 'AF4', 
    'F7', 'F3', 'Fz', 'F4', 'F8',
    'FC5', 'FC1', 'FC2', 'FC6',
    'T7', 'C3', 'Cz', 'C4', 'T8',
    'CP5', 'CP1', 'CP2', 'CP6',
    'P7', 'P3', 'Pz', 'P4', 'P8',
    'PO3', 'PO4', 
    'O1', 'Oz', 'O2'
]
    
# Create standard electrode positions - using the 1005 system montage

montage = mne.channels.make_standard_montage('standard_1005')
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')

# Check whether the channels in montage match the info
montage_ch_names = set(montage.ch_names)
info_ch_names = set(info.ch_names)
info.set_montage(montage)

# 3. Create an Epochs object
n_epochs = eeg_data.shape[0]
events = np.zeros((n_epochs, 3), dtype=int)
events[:, 0] = np.arange(n_epochs) * (eeg_data.shape[1] + 100)
events[:, 2] = 1

# create EpochsArray
epochs = mne.EpochsArray(
    data=eeg_data,
    info=info,
    events=events,
    tmin=tmin,
    event_id={'condition': 1},
    baseline=(-0.2, 0) if tmin <= -0.2 else None,
    proj=False
)
epochs.set_eeg_reference(ref_channels='average', projection=True)

epochs.apply_proj()

print("\nActivated projector:", [p for p in epochs.info["projs"] if p["active"]])

print("\nFinal channel information:")
print(f"The number of channels in the data: {epochs.info['nchan']}")
print(f"channel name: {epochs.info['ch_names']}")

In [ ]:
import numpy as np
import scipy.io
import mne
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse
import matplotlib.pyplot as plt

# 1. load MATLABdata
mat_data = scipy.io.loadmat('name.mat')
eeg_data = mat_data['data']
eeg_data = np.transpose(eeg_data, (2, 0, 1))

# 2. Create the MNE data structure
sfreq = 500 
tmin = 0 

ch_names_64 = [
    'Fp1', 'Fpz', 'Fp2', 
    'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2', 'Iz'
]

ch_names_32 = [
    'Fp1', 'Fp2', 
    'AF3', 'AF4', 
    'F7', 'F3', 'Fz', 'F4', 'F8',
    'FC5', 'FC1', 'FC2', 'FC6',
    'T7', 'C3', 'Cz', 'C4', 'T8',
    'CP5', 'CP1', 'CP2', 'CP6',
    'P7', 'P3', 'Pz', 'P4', 'P8',
    'PO3', 'PO4', 
    'O1', 'Oz', 'O2'
]

# Create a 64-channel data array
n_epochs, n_channels_32, n_times = eeg_data.shape
eeg_data_64 = np.zeros((n_epochs, len(ch_names_64), n_times))

# Map the 32-channel data to the corresponding position in the 64-channel
for i, ch in enumerate(ch_names_32):
    if ch in ch_names_64:
        idx_64 = ch_names_64.index(ch)
        eeg_data_64[:, idx_64, :] = eeg_data[:, i, :]

montage = mne.channels.make_standard_montage('standard_1005')
info = mne.create_info(ch_names=ch_names_64, sfreq=sfreq, ch_types='eeg')
info.set_montage(montage)

# 3. Create an Epochs object
n_epochs = eeg_data_64.shape[0]
events = np.zeros((n_epochs, 3), dtype=int)
events[:, 0] = np.arange(n_epochs) * (eeg_data_64.shape[2] + 100) 
events[:, 2] = 1 

# create EpochsArray
epochs = mne.EpochsArray(
    data=eeg_data_64,
    info=info,
    events=events,
    tmin=tmin,
    event_id={'condition': 1},
    baseline=(-0.2, 0) if tmin <= -0.2 else None,
    proj=False
)

epochs.set_eeg_reference(ref_channels='average', projection=True)
epochs.apply_proj()
print("\nfinal channel info:")
print(f"channel numbers: {epochs.info['nchan']}")
print(f"channel names: {epochs.info['ch_names']}")

# 4. Get the template brain file
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = fs_dir.parent
subject = 'fsaverage'

subjects_dir = fetch_fsaverage(verbose=True).parent
mne.datasets.fetch_fsaverage(subjects_dir)

# 5. Create a forward model (using a template)
src = mne.setup_source_space(
    subject, 
    spacing='ico4',
    subjects_dir=subjects_dir,
    add_dist=False
)

# Three-layer conductivity model
conductivity = (0.3, 0.006, 0.3)
model = mne.make_bem_model(
    subject=subject,
    ico=4,
    conductivity=conductivity,
    subjects_dir=subjects_dir
)
bem = mne.make_bem_solution(model)

trans = mne.read_trans(f'{subjects_dir}/fsaverage/bem/fsaverage-trans.fif')

# Create a positive solution
fwd = mne.make_forward_solution(
    epochs.info,
    trans=trans,
    src=src,
    bem=bem,
    eeg=True,
    mindist=5.0,
    n_jobs=1
)

# 6. Calculate the noise covariance
noise_cov = mne.compute_covariance(
    epochs,
    tmin=tmin, 
    tmax=0.0,
    method='shrunk'
)

# 7. Create the inverse operator
inv = make_inverse_operator(epochs.info, fwd, noise_cov)

# 8. Apply dSPM source localization
epoch = epochs[0]
stc = apply_inverse(epoch, inv, method='dSPM')

# 9. Visualize the source location results
brain = stc.plot(subject='fsaverage', subjects_dir=subjects_dir, 
                hemi='both', views='lateral', time_viewer=True)
brain.add_annotation('aparc', borders=True)

plt.show()

In [ ]:
epochs.plot_sensors(show_names=True, kind='topomap', 
                   title='EEG channel location (standard_1005)',
                   ch_groups='position',
                   
                   show=True)
plt.show()

In [ ]:


# 3. Get the template brain file
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = fs_dir.parent
subject = 'fsaverage'


subjects_dir = fetch_fsaverage(verbose=True).parent
mne.datasets.fetch_fsaverage(subjects_dir)


In [ ]:
# 4.Create a forward model (using a template)
src = mne.setup_source_space(
    subject, 
    spacing='ico4',
    subjects_dir=subjects_dir,
    add_dist=False
)

#conductivity = (0.3,)
conductivity = (0.3, 0.006, 0.3)
model = mne.make_bem_model(
    subject=subject,
    ico=4,
    conductivity=conductivity,
    subjects_dir=subjects_dir
)
bem = mne.make_bem_solution(model)

#trans = mne.read_trans(f'{fs_dir}/fsaverage-trans.fif')
trans = mne.read_trans(f'C:/Users/39397/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-trans.fif')


fwd = mne.make_forward_solution(
    info,
    trans=trans,
    src=src,
    bem=bem,
    eeg=True,
    mindist=5.0,
    n_jobs=1
)

# 5. Calculate the noise covariance
noise_cov = mne.compute_covariance(
    epochs,
    tmin=tmin,
    tmax=0.0,
    method='shrunk'
)

# 6. Create the inverse operator
inv = make_inverse_operator(
    info,
    fwd,
    noise_cov,
    loose=0.2,
    depth=0.8
)

# 7.Apply dSPM to the mean evoked potential
evoked = epochs.average()
stc = apply_inverse(
    evoked,
    inv,
    lambda2=1./9.,
    method='dSPM'
)

fig, ax = plt.subplots()
ax.plot(1e3 * stc.times, stc.data[::100, :].T)
#ax.set(xlabel="time (ms)", ylabel=f"{method} value")

In [ ]:
# 8. Visualization result
brain = stc.plot(
    subjects_dir=subjects_dir,
    subject=subject,
    hemi='both',
    initial_time=0.1,
    clim=dict(kind='percent', lims=[90, 95, 99]),
    views=['lateral', 'medial'],
    size=(800, 600)
)
stc.save('dSPM_source_estimate', overwrite=True)
brain.save_image('source_activation.png', overwrite=True)

plt.figure(figsize=(10, 6))
stc.plot()
plt.title('dSPM Source Time Course')
plt.savefig('source_time_course.png')